# エピソディックで有限ホライゾンなRLの上界と下界

**参考**
* [Sample Complexity of Episodic Fixed-Horizon Reinforcement Learning](https://arxiv.org/abs/1510.08906)

エピソディックで有限ホライゾンな問題設定は現実世界にもよく出てきます。例えば１ヶ月毎にある数学のテストを考えてみます。このときホライゾンは$30$日で、毎月繰り返すのでエピソディックな問題設定です。

TODO: UCRLの証明のほうがわかりやすいかも。Csaba講義参照。


**表記**

定常な有限ホライゾンを考えます。報酬だけ非定常です。

1. 有限状態集合: $S=\{1, \dots, |S|\}$
2. 有限行動集合: $A=\{1, \dots, |A|\}$
3. 遷移確率行列: $p\in \mathbb{R}^{SA\times S}$
4. $t$ステップ目の報酬行列: $r_t\in \mathbb{R}^{S\times A}$
5. ホライゾン: $H$

このとき、エピソードを何回繰り返せば$R_M^*-\epsilon>R_M^\pi$が保証できるか？というPAC RL問題を考えます。

* $\hat{M}$：empiricalなMDP
  * 遷移確率は$\hat{p}$
* $\tilde{M}$：optimisticなMDP
  * 遷移確率は$\tilde{p}_t$
* $P_i^\pi f(s):=\mathbb{E}\left[f\left(s_{i+1}\right) \mid s_i=s\right]=\sum_{s^{\prime} \in \mathcal{S}} p\left(s^{\prime} \mid s, \pi_i(s)\right) f\left(s^{\prime}\right)$
* $P_{i: j}^\pi f:=P_i^\pi P_{i+1}^\pi \ldots P_j^\pi f$
* $V_{i: j}^\pi(s):=\mathbb{E}\left[\sum_{t=i}^j r_t\left(s_t\right) \mid s_i=s\right]=\sum_{t=i}^j P_{i: t-1}^\pi r_t=\left(P_i^\pi V_{i+1: j}^\pi\right)(s)+r_i(s)$
* $\mathcal{S}(s, a) \subseteq \mathcal{S}$：$(s, a)$のあとに続く可能性がある状態の集合
  * $C=\max _{s, a \in \mathcal{S} \times \mathcal{A}}|\mathcal{S}(s, a)|$


## UCFHアルゴリズム

表記：
* $\hat{M}_k$：MDPの最尤推定
* $\mathcal{M}_k$：MDPの信頼集合。UCFHは$\mathcal{M}_k$をもとに方策$\pi^k$を計算します。

UCFHはoptimismを使ってexplorationとexploitationを調整するアルゴリズムです。

![UCFH](figs/UCFH.png)

### 信頼集合

$\operatorname{ConfidenceSet}(p, n)$関数を次の信頼集合を返す関数とします。

$$
\begin{gathered}
\mathcal{P}:=\left\{p^{\prime} \in[0,1]: \text { if } n>1:\left|\sqrt{p^{\prime}\left(1-p^{\prime}\right)}-\sqrt{p(1-p)}\right| \leq \sqrt{\frac{2 \ln \left(6 / \delta_1\right)}{n-1}},\right. \\
\left.\left|p-p^{\prime}\right| \leq \min \left(\sqrt{\frac{\ln \left(6 / \delta_1\right)}{2 n}}, \sqrt{\frac{2 p(1-p)}{n} \ln \left(6 / \delta_1\right)}+\frac{7}{3(n-1)} \ln \frac{6}{\delta_1}\right)\right\}
\end{gathered}
$$

この$\operatorname{ConfidenceSet}$を使って、Optimisticな$\pi^k$は次のExtended Value Iterationで計算します。

### EVIアルゴリズム

Extended Value Iterationの元論文（多分）：
* [An analysis of model-based Interval Estimation for Markov Decision Processes](https://www.sciencedirect.com/science/article/pii/S0022000008000767)

表記：
* optimisticなMDP：$\tilde{M}_k \in \mathcal{M}_k$
* $R_{\tilde{M}_k}^{\pi^k}=\max _{\pi, M^{\prime} \in \mathcal{M}_k} R_{M^{\prime}}^\pi$を見つけるのがEVIの目的

$\mathcal{M}_k$について見つけるのは大変なので、次のMDPの信頼集合についてのoptimisticな方策を見つけます：
$$
\begin{aligned}
\mathcal{M}_k^{\prime}:= & \left\{\tilde{M} \in \mathcal{M}_{\text {nonst. }}: \forall(s, a) \in \mathcal{S} \times \mathcal{A}, t=1 \ldots H, s^{\prime} \in \mathcal{S}(s, a)\right. \\
& \left.\tilde{p}_t\left(s^{\prime} \mid s, a\right) \in \operatorname{conv}\left(\text { ConfidenceSet }\left(\hat{p}\left(s^{\prime} \mid s, a\right), n(s, a)\right)\right)\right\}
\end{aligned}
$$
これを使って、
$$R_{\tilde{M}_k}^{\pi^k}=\max _{\pi, M^{\prime} \in \mathcal{M}_k^{\prime}} R_{M^{\prime}}^\pi$$
を見つけるのを考えます。これは信頼集合の凸包を考えるので、$\mathcal{M}_k \subseteq \mathcal{M}_k^{\prime}$を満たします。

このようなQ関数は次のEVIで解くことができます。

$$
\tilde{Q}_{i: H}^*(s, a)=r_i(s)+\max _{\tilde{p}_i \in \mathcal{P}_{s, a}}\left\{\sum_{s^{\prime} \in \mathcal{S}(s, a)} \tilde{p}_i \max _{b \in \mathcal{A}} \tilde{Q}_{i+1: H}^*\left(s^{\prime}, b\right)\right\}
$$

ここで$\mathcal{P}_{s, a}:=\left\{p \in[0,1]^{|\mathcal{S}(s, a)|} \mid\|p\|_1=1, \forall s^{\prime} \in \mathcal{S}(s, a): p\left(s^{\prime}\right) \in\operatorname{conv}\left(\operatorname{ConfidenceSet}\left(\hat{p}\left(s^{\prime} \mid s, a\right), n(s, a)\right)\right)\right\}$としました。

![EVI](figs/EVI.png)

---

**性能の保証**

$\operatorname{FixedHorizonEVI}\mathcal{M_k}$アルゴリズムは$\tilde{M}, \pi^k=\arg \max _{M \in \mathcal{M}_k^{\prime}, \pi} R_M^\pi$を返します。
また、$\mathcal{M}_k \subseteq \mathcal{M}_k^{\prime}$なので、$R_{\tilde{M}}^{\pi^k} \geq \max _{M \in \mathcal{M}_k, \pi} R_M^\pi$を満たします。

---


## Upper Boundの証明

UCFHアルゴリズムの上界を証明します。

**ステップ１**

UCFHの方策の更新回数が$U_{\max }=|\mathcal{S} \times \mathcal{A}| \log _2 \frac{|\mathcal{S}| H}{w_{\min }}$で抑えられることを確認します。
まず、$n(s, a)$は非減少であり、全ての$(S, a)$で$n(s, a) \geq |S|mH$なら更新は発生しません。

ここで、アルゴリズム中で、SampleEpisodeのところをよく見てみましょう。
* there is a $(s, a) \in \mathcal{S} \times \mathcal{A}$ with $v(s, a) \geq \max \left\{m w_{\min }, n(s, a)\right\}$ and $n(s, a)<|\mathcal{S}| m H$;

の記載から、$\max \left\{m w_{\min }, n(s, a)\right\}$だけ$n(s,a)$が増えた$(s, a)$が一つだけ存在します。
ここで、$\max \left\{m w_{\min }, n(s, a)\right\}$より、$n(s, a)$は一回につき２倍もしくは２倍＋$m w_{\min }-n(s, a)$で増えていきます（最低でも$m w_{\min }$は増えます）。
よって$(s, a)$での更新は$\log _2(|\mathcal{S}| m H)- \log_2(mw_{\min})$回だけ起きます。

以上より、全部で$|\mathcal{S} \times \mathcal{A}| \log _2 \frac{|\mathcal{S}| m H}{w_{\min } m}$回更新が起きます。


**ステップ２** $M \in \mathcal{M}_k$が確率$1-{\delta}/{2}$以上で全ての$k$について成立することを示す。

まずHoeffdingの不等式より
$$\left|p\left(s^{\prime} \mid s, a\right)-\hat{p}\left(s^{\prime} \mid s, a\right)\right| \leq \sqrt{\frac{\ln \left(6 / \delta_1\right)}{2 n}}$$
が確率$1-\delta_1/3$以上で成立。また、Bernsteinより
$$
\left|p\left(s^{\prime} \mid s, a\right)-\hat{p}\left(s^{\prime} \mid s, a\right)\right| \leq \sqrt{\frac{2 p\left(s^{\prime} \mid s, a\right)\left(1-p\left(s^{\prime} \mid s, a\right)\right) \ln \left(6 / \delta_1\right)}{n}}+\frac{1}{3 n} \ln \left(6 / \delta_1\right)
$$
が確率$1-\delta_1/3$以上で成立。
よって、

$$
\mathbb{P}\left(\left|P\left(s^{\prime} \mid s, a\right)-\widehat{P}\left(s^{\prime} \mid s, a\right)\right| \leq \min \left\{\sqrt{\frac{2 \widehat{P}\left(s^{\prime} \mid s, a\right)\left(1-\widehat{P}\left(s^{\prime} \mid s, a\right)\right)}{n(s, a)} \log \frac{4}{\delta_1}}+\frac{2}{3 n(s, a)} \log \frac{4}{\delta_1}, \sqrt{\frac{\log 4 / \delta_1}{2 n(s, a)}}\right\}\right) \geq 1-\delta_1
$$

が成立する。

ここで、Empirical Bernstein（[Empirical Bernstein Bounds and Sample Variance Penalization](https://arxiv.org/abs/0907.3740)）より

$$
\left|\sqrt{p\left(s^{\prime} \mid s, a\right)\left(1-p\left(s^{\prime} \mid s, a\right)\right)}-\sqrt{\hat{p}\left(s^{\prime} \mid s, a\right)\left(1-\hat{p}\left(s^{\prime} \mid s, a\right)\right)}\right| \leq \sqrt{\frac{2 \ln \left(6 / \delta_1\right)}{n-1}}
$$

が確率$1-\delta_1/3$以上で成立する。

よって、
$$
\begin{aligned}
\left|p\left(s^{\prime} \mid s, a\right)-\hat{p}\left(s^{\prime} \mid s, a\right)\right| & \leq \sqrt{\frac{2 p\left(s^{\prime} \mid s, a\right)\left(1-p\left(s^{\prime} \mid s, a\right)\right) \ln \left(6 / \delta_1\right)}{n}}+\frac{1}{3 n} \ln \left(6 / \delta_1\right) \\
& \leq\left(\sqrt{\hat{p}\left(s^{\prime} \mid s, a\right)\left(1-\hat{p}\left(s^{\prime} \mid s, a\right)\right)}+\sqrt{\frac{2 \ln \left(6 / \delta_1\right)}{n-1}}\right) \sqrt{\frac{2 \ln \left(6 / \delta_1\right)}{n}}+\frac{1}{3 n} \ln \left(6 / \delta_1\right) \\
& \leq \sqrt{\frac{2 \hat{p}\left(s^{\prime} \mid s, a\right)\left(1-\hat{p}\left(s^{\prime} \mid s, a\right)\right) \ln \left(6 / \delta_1\right)}{n}}+\frac{7}{3(n-1)} \ln \left(6 / \delta_1\right) .
\end{aligned}
$$

であり、ステップ１より全部で$U_{\max}$回の更新しか起きないので、$k$についてのUnion Boundは$U_{\max}$回だけ取れば良い。
また、$C=\max _{s, a \in \mathcal{S} \times \mathcal{A}}|\mathcal{S}(s, a)|$より、$s'$の種類は$C$個しかないので、UnionBoundは$U_{\max}C$について取れば良く、$\delta_1=\frac{\delta}{2 C U_{\max }}$とすれば証明終わり。


**ステップ３**

PAC RLのサンプル効率の導出では「knownness」という二値判定を使って、遷移確率が十分正確に推定されているかどうかを判定する手法がしばしば取られます（論文の５ページめあたりに参考文献がまとまっています）。
このknownnessを使ってバウンドを導出していきます。

証明のために次の表記を導入します。まず、$(s, a)$の「weight」を次で定義される期待頻度とします：

$$
w_k(s, a):=\sum_{t=1}^H \mathbb{P}\left(s_t=s, \pi_t^k\left(s_t\right)=a\right)=\sum_{t=1}^H P_{1: t-1} \mathbb{I}\left\{s=\cdot, a=\pi_t^k(s)\right\}\left(s_0\right) .
$$

また、次の「importance」の概念を導入します：
$$
\iota_k(s, a):=\min \left\{z_i: z_i \geq \frac{w_k(s, a)}{w_{\min }}\right\} \quad \text { where } z_1=0 \text { and } z_i=2^{i-2} \forall i=2,3, \ldots
$$
ここで、$w_{\min }:=\frac{\epsilon}{4 H|S|}$、$z_1=0$、$z_j=2^{j-2} \forall j=2,3, \ldots$とします。
$\iota_k(s, a) \in\{0,1,2,4,8,16, \ldots\}$は$w_{\min}$に対して訪問する確率のスケールを表します。
「importanceが高い」$\approx$「$0\sim H$で訪れる確率が高い」ことになるので、importanceは$(s, a)$が価値観数に与える影響を表しています。

最後に「knownness」の概念を導入します：
$$
\kappa_k(s, a):=\max \left\{z_i: z_i \leq \frac{n_k(s, a)}{m w_k(s, a)}\right\} \in\{0,1,2,4, \ldots\}
$$
$m$は後で出てくる定数です。
$(s, a)$が訪問された回数を重要度でスケールした量です。「knownnessが高い」$\approx$「importanceが低い　もしくは　頻繁に訪れた」ことになります。

これを使って、$(s, a)$のペアを次の部分集合に分割します：
$$
X_{k, \kappa, \iota}:=\left\{(s, a) \in X_k: \kappa_k(s, a)=\kappa, \iota_k(s, a)=\iota\right\} \quad \text { and } \quad \bar{X}_k=\mathcal{S} \times \mathcal{A} \backslash X_k
$$
ここで、$X_k=\left\{(s, a): \iota_k(s, a)>0\right\}$としました。つまり、$X_k$は$0\sim H$で訪れる確率が０以上の状態と行動の集合です。
よって、$\bar{X}_k=S \times A \backslash X_k$は訪れる可能性がほぼない状態と行動の集合です。

---

**直感的な解釈**

「UCFHが正確」$\approx$ 「knownnessが低い$(s, a)$の個数が少ない」$\approx$ 「importanceが高いが、それほど観測していない$(s, a)$の個数が少ない」

ことを示していきます。
そのために、UCFHについて、一定以上のエピソードになると
$$
\left|X_{k, \kappa, \iota}\right| \leq \kappa
$$
が任意の$\kappa$と$\iota$について成立する、ということを証明していきます。
後で証明しますが、$\left|X_{k, \kappa, \iota}\right| \leq \kappa$が任意の$\kappa$と$\iota$について成立すると、
$$
\left|\tilde{V}_{1: H}^{\pi^k}\left(s_0\right)-V_{1: H}^{\pi^k}\left(s_0\right)\right| \leq \epsilon
$$
が証明できます。まとめると、次の方針で証明を進めていきます：

1. 真のMDP $M$ が$\mathcal{M}_k$に確率$1-\delta/2$で入ることを証明します（ステップ２　）
2. FixedHorizonEVIが返す$\tilde{M}$と$\pi^k$について、$\tilde{M}, \pi^k=\arg \max _{M \in \mathcal{M}_k^{\prime}, \pi} R_M^\pi$を証明します。これは$\mathcal{M}_k \subseteq \mathcal{M}_k^{\prime}$なので、$R_{\tilde{M}}^{\pi^k} \geq \max _{M \in \mathcal{M}_k, \pi} R_M^\pi \geq \max_{\pi}R_{M}^{\pi} $を満たします。（省略）
3. $\left|X_{k, \kappa, \iota}\right|>\kappa$となる$\kappa$と$\iota$が高い確率で存在するエピソード$k$は$\tilde{O}(|\mathcal{S} \times \mathcal{A}| m)$でバウンドできることを示します。
4. $\left|X_{k, \kappa, \iota}\right| \leq \kappa$が任意の$\kappa$と$\iota$で成立するとき、つまり、小さいknownnessの状態行動数が小さい数で抑えられるとき、楽観的な価値が真のMDPでの価値と$\epsilon$のオーダーで近いことを示します
5. 3と合わせて、$\epsilon$最適な価値に至るエピソードの数をバウンドします。

---

これを踏まえて、３番を証明しましょう。つまり、$\left|X_{k, \kappa, \iota}\right|>\kappa$となる$\kappa$と$\iota$が高い確率で存在するエピソード$k$は$\tilde{O}(|\mathcal{S} \times \mathcal{A}| m)$でバウンドできることを示します。


**ステップ３の続き** 

まず、$\kappa \in[1,|\mathcal{S}|-1]$ かつ $\iota>0$については、$(s, a) \in X_{k, \kappa, \iota}$の要素を訪れた回数全体は、最大でも$3|\mathcal{S}\times \mathcal{A}|m w_{k, \iota} \kappa$であることを示します。
ここで$w_{k, \iota} := \min_{(s, a) \in (\mathcal{S}\times \mathcal{A})} \left\{w_k(s, a): \iota_k(s, a)=\iota\right\}$とします。

ここで、
$$
w_{k, 2\iota}=\min_{(s, a)} \left\{w_k(s, a): \min_j \left\{z_j=2^{j-2}: z_j \geq \frac{w_k(s, a)}{w_{\min }}\right\}=2\iota\right\}
$$
をよく見てみます。$w_k(s, a)$の値は$w_\iota$と$w_{2\iota}$で変化しないことに注意すると、
$$
w_{k, 2\iota}=\min_{(s, a)} \left\{w_k(s, a): \min_j \left\{z_j=2^{j-2}: z_j \geq \frac{w_k(s, a)}{w_{\min }}\right\}=2\iota\right\} = 2w_{k, \iota}
$$
が成立します。

続いて、固定された$k$と$(s, a) \in X_{k, \kappa, \iota}$について考えます。
任意の$(s, a) \in X_{k, \kappa, \iota}$については$\iota_k(s, a)= \min_j \left\{z_j=2^{j-2}: z_j \geq \frac{w_k(s, a)}{w_{\min }}\right\}=\iota>0$なので、

$$
w_\iota \leq w_k(s, a)<2 w_\iota
$$

が成り立ちます。（下の図参照。）
注意として、ここでは$w_{2\iota}$は存在しない場合があります。しかし$2 w_\iota$でバウンドしているので、$w_\iota$の存在は仮定しているため大丈夫です。

![tmp](figs/episodic_finite_tmp.jpg)

よって、$\kappa_k(s, a)=\kappa$なので、同様にして、

$$
\frac{n_k(s, a)}{2 m w_k(s, a)} \leq \kappa \leq \frac{n_k(s, a)}{m w_k(s, a)}
$$

が成立します。これより、

$$
m w_\iota \kappa \leq m w_k(s, a) \kappa \leq n_k(s, a) \leq 2 m w_k(s, a) \kappa \leq 4 m w_\iota \kappa .
$$

であり、$\left\{(s, a) \in X_{k, \kappa, \iota}: k \in \mathbb{N}\right\}$では状態行動対は$4mw_\iota \kappa$回しか観測されません。

**ステップ４の続き** 

$\left|X_{k, \kappa, \iota}\right|>\kappa$となるエピソードの数$E_{\kappa, \iota}$が

$$P\left(E_{\kappa, \iota}>\alpha N\right) \leq \exp \left(-\frac{\beta w_\iota(\kappa+1) N}{H}\right)$$

でバウンドされることを示します。ここで、$\alpha \geq 3$、$N=|\mathcal{S} \times \mathcal{A}| m$ そして $\beta=\frac{\alpha(3 / \alpha-1)^2}{7 / 3-1 / \alpha}$です。
注意として、エピソードの数 $\neq$ phaseの数(while loopの回数)です（よく見るとwhile loopの中で何度もエピソードを繰り返すことがあります）

$\nu_i:=\sum_{t=1}^H \mathbb{I}\left\{\left(s_t, a_t\right) \in X_{k, \kappa, \iota}\right\}$を、$X_{k, \kappa, \iota}>\kappa$を満たす$i$番目のエピソードでの$X_{k, \kappa, \iota}$内の$(s, a)$を観測した数とします。

$\left|X_{k, \kappa, \iota}\right|>\kappa$となるエピソードの数が$E_{\kappa, \iota}$なので、$i \in\left\{1, \ldots E_{\kappa, \iota}\right\}$であり、$k$は$i$が所属するphaseとします。

$\kappa \in \{0, 1, 2, 4, \cdots\}$なので、$|X_{k, \kappa, \iota}| \geq \kappa+1$であり、さっきの証明で見たように$w_k(s, a) \geq w_\iota$なので、

$$\mathbb{E}\left[\nu_i \mid \nu_1, \ldots \nu_{i-1}\right] \geq(\kappa+1) w_\iota$$

が成り立ちます（ややこしいですが定義を確認するとあってそうですね）。
ここで、$\nu_i \in[0, H]$なので、$\mathbb{V}\left[\nu_i \mid \nu_1 \ldots \nu_{i-1}\right] \leq \mathbb{E}\left[\nu_i \mid \nu_1, \ldots \nu_{i-1}\right] H$ がなりたちます。

以下、次の表記を導入します。
$$
\nu_i^{+}:= \begin{cases}\nu_i & \text { if } i \leq E_{\kappa, \iota} \\ w_\iota(\kappa+1) & \text { otherwise }\end{cases}
$$

また、
$$
\bar{\nu}_i:=\frac{\nu_i^{+} w_\iota(\kappa+1)}{\mathbb{E}\left[\nu_i^{+} \mid \nu_1^{+}, \ldots \nu_{i-1}^{+}\right]}
$$
とします。

このとき、

$$
\mathbb{E}\left[\bar{\nu}_i \mid \bar{\nu}_1, \ldots \bar{\nu}_{i-1}\right]=w_\iota(\kappa+1) \frac{\mathbb{E}\left[\nu_i^{+} \mid \bar{\nu}_1, \ldots, \bar{\nu}_{i-1}\right]}{\mathbb{E}\left[\nu_i^{+} \mid \nu_1^{+}, \ldots \nu_{i-1}^{+}\right]}=w_\iota(\kappa+1)
$$

が成り立っています。ここで、上でやった定理から、$E_{\kappa, \iota > \alpha N}$になるのは
$\sum_{i=1}^{\alpha N} \bar{\nu}_i \leq 3 N w_\iota \kappa \leq 3 N w_\iota(\kappa+1)$
のときだけです。
